In [1]:
import importlib
import numpy as np
import sklearn
from matplotlib import pyplot as plt
from os.path import join
import os
import seaborn as sns
from torchvision.ops.misc import interpolate
from tqdm.notebook import tqdm as tqdm_notebook

#### Custum libraries
import lib.algos_maxRSA as max_rsa
import lib.utils_RSA as rsa
import lib.utils_CKA as cka
from lib.algos import *
from scipy.spatial import procrustes as scipro


importlib.reload(rsa)
importlib.reload(cka)
importlib.reload(max_rsa)

<module 'lib.algos_maxRSA' from '/home/alban/projects/SAYCam_Vs_EGO4D/lib/algos_maxRSA.py'>

In [2]:
dataset = 'ecoVal'
models  = ['ego', 'saycam', 'imagenet', 'supervised', 'random', 'resnet']
#models  = ['ego', 'saycam']
path2activations = f'/home/alban/Documents/activations_datadriven/%s_{dataset}/'

imagelists = {}
activations = {}
for model in models:
    with open(join(path2activations%model, 'imagepaths.txt'), 'r') as f:
        imagelists[model] = [line.strip() for line in f.readlines()]
    activations[model] = np.load(join(path2activations % model, 'cls_tokens.npy'))

activations[model].shape

(28250, 2048)

In [3]:
#### Normalize vectors
for model in models:
    norms = np.linalg.norm(activations[model], axis=1, keepdims=True)
    activations[model] = activations[model]/norms # normalization

In [4]:
#### Compute RDMs performing squared euclidean distance as a metric (to be equivalent to CKA methods, cf. Williams, 2024)
RDMs = {}
submodels = ['ego', 'saycam', 'imagenet', 'supervised', 'resnet']
metric = 'L2squared'
for i, model in enumerate(submodels):
    print(model)
    RDMs[model] = rsa.compute_RDMs(activations[model], metric = metric, display = False, title = f'{model}_{metric}')
    #RDMs[model] = cka.centering(RDMs[model]) # same as centering with matmul


ego
saycam
imagenet
supervised
resnet


In [5]:
### check if images were shown in the same order
assert imagelists['ego'] == imagelists['saycam']
imagelist = imagelists['ego'] # since they are the same, only consider one list

#### check if each category has the same number of images and list all categories in listcats
count = 0
cat = ''
listcat = list()
for i, imgp in enumerate(imagelist):
    current_cat = imgp.split('/')[-2]
    if i == 0:
        cat = current_cat
        listcat.append(current_cat)
    if cat != current_cat:
        cat = current_cat
        listcat.append(current_cat)
        count = 1
    else:
        count += 1

nb_per_cat = count # in val, 50 images per cate

nb_per_cat
#np.array(imagelist).reshape(-1, nb_per_cat)[0]

50

In [6]:
### reshape activations according to include categories
cat_activations = activations.copy()

for model in models:
    shape = activations[model].shape
    cat_activations[model] = activations[model].reshape(-1, nb_per_cat, shape[-1])

In [7]:
### Compute representational compactness for each category and model
compactness, compact_categories = max_rsa.compute_Fisher_descriminant(cat_activations, models, listcat)


AttributeError: module 'lib.algos_maxRSA' has no attribute 'compute_Fisher_descriminant'

In [ ]:
max_rsa.plot_stats_one(compactness,models,  ['Categories', 'Normalized var'], savename='Fisher_descriminant_allmodels.png')
for model in models:
    print(f'{model}')
    print(compact_categories[model][:20])
for model in models:
    print(compact_categories[model][-20:])


In [ ]:
def check_list_similarity(list1, list2):
    '''Checks if two lists contain the same elements, regardless of order,
    and calculates the proportion of common elements.'''
    set1 = set(list1)
    set2 = set(list2)
    common_elements = set1 & set2  # Intersection of sets
    proportion = (len(common_elements) / max(len(set1), len(set2))) * 100 if max(len(set1), len(set2)) > 0 else 0
    return proportion


for model in models:
    print(f'\n Overlap {model} and the others')
    for model2 in models[:]:
        print(check_list_similarity(compact_categories[model][:50],compact_categories[model2][:50]))


In [ ]:
### Compute 500 random sequence for significance
# initialize sequences
nb_trials = 500
idx_vec = np.array(range(len(listcat)))
mat_vec = np.zeros((nb_trials, len(idx_vec)))
for i in range(nb_trials):
    np.random.shuffle(idx_vec)
    mat_vec[i] = idx_vec

#Compute all possible similarity pairs
list_sim = []
for i in range(len(mat_vec)-1):
    for j in range(i+1,len(mat_vec)):
        list_sim.append(check_list_similarity(list(mat_vec[i][:50]), list(mat_vec[j][:50])))

# Compute 95 percentile
confinter = np.percentile(list_sim, 95)
print(f'The 95% confidence interval is:{confinter}')

In [ ]:
nb_categories = len(listcat)
labels, sortedmaxdiffcats, maxdiffs = max_rsa.max_compactness_difference(compact_categories, compactness, nb_categories, listcat, models = ['ego', 'saycam'], nb_max_compactness = 20)

In [ ]:
from itertools import combinations
def find_max_dissimilarity_images(RDMs, models, categories, nb_per_cat,
                                  images_per_subset=4):
    """
    Find the subset of images per category that maximizes RDM dissimilarity between two models.

    Parameters:
    -----------
    cat_activations : dict
        Dictionary with structure: cat_activations[model][category] = array of activations (n_images, n_features)
    models : list
        List of two model names, e.g., ['model1', 'model2']
    categories : list
        List of category names/indices
    compute_RDM : function
        Function that takes activations and returns RDM: RDM = compute_RDM(activations)
    images_per_subset : int
        Number of images to select per category (default: 4)
    method : str
        'exhaustive' or 'random' sampling of combinations

    Returns:
    --------
    results : dict
        Dictionary with results for each category:
        {
            category: {
                'best_indices': array of selected image indices,
                'max_dissimilarity': maximum dissimilarity value,
                'model1_rdm': RDM for model1 with selected images,
                'model2_rdm': RDM for model2 with selected images,
                'similarity': similarity between the two RDMs
            }
        }
    """

    if len(models) != 2:
        raise ValueError("This function requires exactly 2 models")

    results = {}

    means = {}
    n = len(RDMs[models[0]])
    upper_indices = np.triu_indices(n, k=1)  # k=1 excludes diagonal
    means['x'] = np.mean(RDMs[models[0]][upper_indices])
    means['y'] = np.mean(RDMs[models[1]][upper_indices])
    means['norm'] = np.std(RDMs[models[0]][upper_indices]) * np.std(RDMs[models[1]][upper_indices])
    print(means)
    for category in tqdm_notebook(categories, desc="Processing categories"):
        print(f"\nProcessing category: {category}")
        # Get activations for both models for this category
        RDM1 = RDMs[models[0]][category*nb_per_cat:(category+1)*nb_per_cat, category*nb_per_cat:(category+1)*nb_per_cat]  # Shape: (50, 50)
        RDM2 = RDMs[models[1]][category*nb_per_cat:(category+1)*nb_per_cat, category*nb_per_cat:(category+1)*nb_per_cat]  # Shape: (50, 50)

        # Generate combinations of image indices
        all_combinations = list(combinations(range(nb_per_cat), images_per_subset))

        print(f"Testing {len(all_combinations)} combinations of {images_per_subset} images")

        best_indices = None
        best_model1_rdm = None
        best_model2_rdm = None
        best_similarity = 1

        # Test each combination
        for combination in tqdm_notebook(all_combinations, desc="Testing combinations", leave=False, position=1):
            indices = np.array(combination)
            # Get subset of activations
            rdm1 = RDM1[np.ix_(indices, indices)]  # Shape: (4, 4)
            rdm2 = RDM2[np.ix_(indices, indices)]  # Shape: (4, 4)

            # Compute similarity between RDMs

            similarity = rsa.Compute_sim_RDMs(rdm1, rdm2, center = False, metric = 'pearson_global', means= means)

            # Update best if this is better
            if similarity < best_similarity:
                best_indices = indices
                best_model1_rdm = rdm1
                best_model2_rdm = rdm2
                best_similarity = similarity

        # Store results for this category
        results[category] = {
            'best_indices': best_indices,
            'model1_rdm': best_model1_rdm,
            'model2_rdm': best_model2_rdm,
            'similarity': best_similarity
        }

        print(f"Best indices for {category}: {best_indices}")
        print(f"Similarity: {best_similarity:.4f}")

    return results



In [ ]:
results = find_max_dissimilarity_images(
        RDMs, ['ego', 'saycam'], labels[:10], 50,
        images_per_subset=4
    )

In [ ]:
def compute_sub_rdm_similarity(results, RDMs, models, categories):
    """
    Compute sub-RDMs using the 40 selected images (4 per category × 10 categories)
    that maximize dissimilarity between models, then compute their similarity.

    Parameters:
    -----------
    results : dict
        Output from find_max_dissimilarity_images() with structure:
        results[category]['best_indices'] = array of 4 selected image indices
    full_rdms : dict
        Dictionary: full_rdms[model] = full RDM array (25000, 25000)
    models : list
        List of two model names, e.g., ['model1', 'model2']
    categories : list
        List of category names/indices (should have 10 categories)

    Returns:
    --------
    result : dict
        Dictionary containing:
        {
            'similarity': similarity between the two 40×40 RDMs,
            'model1_rdm': 40×40 RDM for model1,
            'model2_rdm': 40×40 RDM for model2,
            'image_info': list of (category, original_index) for each of the 40 images
        }
    """

    if len(models) != 2:
        raise ValueError("This function requires exactly 2 models")

    print(f"Collecting 40 selected images from {len(categories)} categories...")

    means = {}
    n = len(RDMs[models[0]])
    upper_indices = np.triu_indices(n, k=1)  # k=1 excludes diagonal
    means['x'] = np.mean(RDMs[models[0]][upper_indices])
    means['y'] = np.mean(RDMs[models[1]][upper_indices])
    means['norm'] = np.std(RDMs[models[0]][upper_indices]) * np.std(RDMs[models[1]][upper_indices])
    # Collect selected image indices from all categories
    selected_indices = []
    image_info = []  # Track which category and original index each image comes from

    total_selected = 0

    for category in categories:
        if category not in results:
            raise ValueError(f"Category {category} not found in results")

        # Get the 4 selected indices for this category
        cat_selected_indices = results[category]['best_indices']

        if len(cat_selected_indices) != 4:
            print(f"Warning: Category {category} has {len(cat_selected_indices)} images instead of 4")

        # Add to combined list
        selected_indices.extend(cat_selected_indices + 50*category)

        # Track image information
        for idx in cat_selected_indices:
            image_info.append((category, idx))

        total_selected += len(cat_selected_indices)
        #print(f"Category {category}: selected indices {cat_selected_indices}")

    print(f"\nTotal selected images: {total_selected}")

    # Verify we have 40 images
    if len(selected_indices) != 40:
        print(f"Warning: Expected 40 images, got {len(selected_indices)}")

    # Extract sub-RDMs for both models using the 40 selected images
    print("Extracting sub-RDMs...")
    rdm_model1 = RDMs[models[0]][np.ix_(selected_indices , selected_indices )]  # Shape: (40, 40)
    rdm_model2 = RDMs[models[1]][np.ix_(selected_indices , selected_indices  )]  # Shape: (40, 40)

    print(f"RDM shapes: {rdm_model1.shape}, {rdm_model2.shape}")

    # Compute similarity between the two RDMs
    print("Computing similarity between RDMs...")
    similarity = rsa.Compute_sim_RDMs(rdm_model1, rdm_model2, center = False, metric = 'pearson')

    print(f"\nRDM similarity using 40 maximally dissimilar images: {similarity:.6f}")

    # Package results
    result = {
        'similarity': similarity,
        'model1_rdm': rdm_model1,
        'model2_rdm': rdm_model2,
        'image_info': image_info,
        'total_images': total_selected,
        'selected_indices': selected_indices
    }

    fig, subs = plt.subplots(1,2)
    sns.heatmap(rdm_model1,
                annot=False,
                cmap='Greys',      # Blue to red colormap
                square=True,
                cbar=True,
                #cbar_kws={'label': 'Dissimilarity'},
                #fmt='.2f',
                linewidths=0,
                ax = subs[0],
                vmin=0,               # Set minimum value for color scale
                vmax=np.max(rdm_model1))               # Set maximum value for color scale
    sns.heatmap(rdm_model2,
                annot=False,
                cmap='Greys',      # Blue to red colormap
                square=True,
                cbar=True,
                #cbar_kws={'label': 'Dissimilarity'},
                #fmt='.2f',
                linewidths=0,
                ax = subs[1],
                vmin=0,               # Set minimum value for color scale
                vmax=np.max(rdm_model2))


    subs[0].axis('off')
    subs[1].axis('off')
    fig.tight_layout()
    plt.show()
    return result

In [ ]:
similarity_dict = compute_sub_rdm_similarity(results, RDMs, ['ego', 'saycam'], labels[:10])


In [ ]:

def sample_rdm_pairs(RDM1, RDM2, n_samples=100000, subset_size=40,
                                    batch_size=10000, seed=None):
    """
    Memory-efficient version that processes in batches and optionally saves to disk.

    Parameters:
    -----------
    batch_size : int
        Number of samples to process at once (default: 1000)
    output_file : str, optional
        If provided, saves results to this file using pickle
    """

    if seed is not None:
        np.random.seed(seed)

    n_images = RDM1.shape[0]
    n_batches = (n_samples + batch_size - 1) // batch_size

    all_sims_samples = []
    all_indices = []
    means = {}
    n = len(RDMs[models[0]])
    upper_indices = np.triu_indices(n, k=1)  # k=1 excludes diagonal
    means['x'] = np.mean(RDM1[upper_indices])
    means['y'] = np.mean(RDM2[upper_indices])
    means['norm'] = np.std(RDM1[upper_indices]) * np.std(RDM2[upper_indices])
    print(f"Processing {n_samples} samples in {n_batches} batches of {batch_size}...")

    for batch_idx in tqdm_notebook(range(n_batches)):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, n_samples)
        current_batch_size = end_idx - start_idx

        # Allocate batch arrays
        batch_sim = np.zeros((current_batch_size))
        batch_indices = np.zeros((current_batch_size, subset_size), dtype=int)

        for i in range(current_batch_size):
            # Randomly select images
            indices = np.random.choice(n_images, size=subset_size, replace=False)
            indices = np.sort(indices)

            # Extract submatrices
            batch_sim[i] = rsa.Compute_sim_RDMs(RDM1[np.ix_(indices, indices)], RDM2[np.ix_(indices, indices)], center = False, metric = 'pearson' )
            batch_indices[i] = indices

        all_sims_samples.append(batch_sim)
        all_indices.append(batch_indices)

    # Concatenate all batches
    sim_samples = np.concatenate(all_sims_samples, axis=0)
    indices_used = np.concatenate(all_indices, axis=0)


    return sim_samples, indices_used




In [ ]:
sim_samples, indices_used = sample_rdm_pairs(RDMs['ego'], RDMs['saycam'], n_samples=1000000, subset_size=40,
                                    batch_size=1000, seed=None)

In [ ]:
hist, bin_edges = np.histogram(sim_samples, 100)
plt.bar(bin_edges[:-1],hist/max(hist), width = bin_edges[1] - bin_edges[0], align = 'edge')
plt.vlines(similarity_dict['similarity'], 0, 1, 'green', label = 'Subset_selected')
plt.vlines(np.mean(sim_samples), 0, 1, 'r', label = 'Average')
plt.legend()
plt.xlabel('Similarity')
plt.ylabel('Density')
plt.show()
perc = np.sum(sim_samples< similarity_dict['similarity'])/len(sim_samples)


In [ ]:
perc*len(sim_samples)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

def display_low_similarity_images(image_paths, indices_vectorized, n_images=40,
                                 grid_cols=8, figsize=(20, 10), save_path=None):
    """
    Load and display the first n images corresponding to lowest similarity indices.

    Parameters:
    -----------
    image_paths : list
        List of image file paths matching the RDM column indices
    indices_vectorized : numpy.ndarray
        Column indices sorted from lowest to highest similarity
    n_images : int
        Number of images to display (default: 40)
    grid_cols : int
        Number of columns in the display grid (default: 8)
    figsize : tuple
        Figure size for matplotlib (default: (20, 10))
    save_path : str, optional
        Path to save the figure (if None, just display)

    Returns:
    --------
    loaded_images : list
        List of loaded images (as numpy arrays)
    valid_paths : list
        List of valid image paths that were successfully loaded
    """

    # Get the indices for the first n_images with lowest similarity
    low_similarity_indices = indices_vectorized[:n_images]

    # Get corresponding image paths
    selected_paths = [image_paths[idx] for idx in low_similarity_indices]

    loaded_images = []
    valid_paths = []
    valid_indices = []

    print(f"Loading {n_images} images with lowest RDM column similarity...")

    # Load images
    for i, (path, orig_idx) in enumerate(zip(selected_paths, low_similarity_indices)):
        try:
            # Check if file exists
            if not os.path.exists(path):
                print(f"Warning: File not found: {path}")
                continue

            # Load image with cv2
            img = cv2.imread(path)

            if img is None:
                print(f"Warning: Could not load image: {path}")
                continue

            # Convert BGR to RGB for matplotlib display
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            loaded_images.append(img_rgb)
            valid_paths.append(path)
            valid_indices.append(orig_idx)

        except Exception as e:
            print(f"Error loading {path}: {e}")
            continue

    print(f"Successfully loaded {len(loaded_images)} out of {n_images} requested images")

    if len(loaded_images) == 0:
        print("No images could be loaded!")
        return [], []

    # Calculate grid dimensions
    n_loaded = len(loaded_images)
    grid_rows = (n_loaded + grid_cols - 1) // grid_cols

    # Create figure and display images
    fig, axes = plt.subplots(grid_rows, grid_cols, figsize=figsize)
    fig.suptitle(f'Images leading to lowest similarity',
                 fontsize=16, y=0.98)

    # Handle case where we have only one row
    if grid_rows == 1:
        axes = axes.reshape(1, -1)

    for i in range(grid_rows * grid_cols):
        row = i // grid_cols
        col = i % grid_cols
        ax = axes[row, col]

        if i < len(loaded_images):
            # Display image
            ax.imshow(loaded_images[i])

            # Add title with original index and filename
            filename = Path(valid_paths[i]).name
            label = valid_paths[i].split('/')[-2]
            ax.set_title(f'Label: {label }\n{filename[:20]}...',
                        fontsize=8, pad=2)

        ax.axis('off')

    plt.tight_layout()

    # Save or show
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Figure saved to: {save_path}")

    plt.show()

    return loaded_images, valid_paths

In [ ]:
imagelist = [img.replace('/raid/shared/datasets/visoin/', '/home/alban/Documents/') for img in imagelist]
images, paths = display_low_similarity_images(imagelist, similarity_dict['selected_indices'], n_images=40,
                                 grid_cols=8, figsize=(20, 10), save_path='figures/images_inverse_Fisher_descriminant.png')

Let's do the same for the same analysis for the models trained on saycam and imagenet.


In [ ]:
labels_say_imagenet, sortedmaxdiffcats_say_imagenet, maxdiffs_say_imagenet = max_rsa.max_compactness_difference(compact_categories, compactness, nb_categories, listcat, models = ['saycam', 'imagenet'], nb_max_compactness = 20)

results_say_imagenet = find_max_dissimilarity_images(
        RDMs, ['saycam', 'imagenet'], labels_say_imagenet[:10], 50,
        images_per_subset=4)



In [ ]:
similarity_dict_say_imagenet = compute_sub_rdm_similarity(results_say_imagenet, RDMs, ['saycam', 'imagenet'], labels_say_imagenet[:10])
imagelist = [img.replace('/raid/shared/datasets/visoin/', '/home/alban/Documents/') for img in imagelist]
images, paths = display_low_similarity_images(imagelist, similarity_dict_say_imagenet['selected_indices'], n_images=40,
                                 grid_cols=8, figsize=(20, 10), save_path='figures/images_inverse_Fisher_descriminant_saycam_imagnet.png')

In [ ]:
sim_samples_say_imagenet, indices_used_say_imagenet = sample_rdm_pairs(RDMs['saycam'], RDMs['imagenet'], n_samples=1000000, subset_size=40, batch_size=1000, seed=None)


In [ ]:
hist, bin_edges = np.histogram(sim_samples_say_imagenet, 100)
plt.bar(bin_edges[:-1],hist/max(hist), width = bin_edges[1] - bin_edges[0], align = 'edge')
plt.vlines(similarity_dict_say_imagenet['similarity'], 0, 1, 'green', label = 'Subset_selected')
plt.vlines(np.mean(sim_samples_say_imagenet), 0, 1, 'r', label = 'Average')
plt.legend()
plt.xlabel('Similarity')
plt.ylabel('Density')
plt.show()
perc = np.sum(sim_samples_say_imagenet< similarity_dict_say_imagenet['similarity'])/len(sim_samples_say_imagenet)

In [ ]:
perc*len(sim_samples_say_imagenet)

Same thing between saycam and the supervised.

In [ ]:
labels_say_supervised, sortedmaxdiffcats_say_supervised, maxdiffs_say_supervised = max_rsa.max_compactness_difference(compact_categories, compactness, nb_categories, listcat, models = ['saycam', 'supervised'], nb_max_compactness = 20)

results_say_supervised = find_max_dissimilarity_images(
        RDMs, ['saycam', 'supervised'], labels_say_supervised[:10], 50,
        images_per_subset=4)



In [ ]:
similarity_dict_say_supervised = compute_sub_rdm_similarity(results_say_supervised, RDMs, ['saycam', 'supervised'], labels_say_supervised[:10])
imagelist = [img.replace('/raid/shared/datasets/visoin/', '/home/alban/Documents/') for img in imagelist]
images, paths = display_low_similarity_images(imagelist, similarity_dict_say_supervised['selected_indices'], n_images=40,
                                 grid_cols=8, figsize=(20, 10), save_path='figures/images_inverse_Fisher_descriminant_saycam_supervised.png')

In [ ]:
sim_samples_say_supervised, indices_used_say_supervised = sample_rdm_pairs(RDMs['saycam'], RDMs['supervised'], n_samples=1000000, subset_size=40, batch_size=1000, seed=None)

In [ ]:
hist, bin_edges = np.histogram(sim_samples_say_supervised, 100)
plt.bar(bin_edges[:-1],hist/max(hist), width = bin_edges[1] - bin_edges[0], align = 'edge')
plt.vlines(similarity_dict_say_supervised['similarity'], 0, 1, 'green', label = 'Subset_selected')
plt.vlines(np.mean(sim_samples_say_supervised), 0, 1, 'r', label = 'Average')
plt.legend()
plt.xlabel('Similarity')
plt.ylabel('Density')
plt.show()
perc = np.sum(sim_samples_say_supervised< similarity_dict_say_supervised['similarity'])/len(sim_samples_say_supervised)

In [ ]:
labels_say_resnet, sortedmaxdiffcats_say_resnet, maxdiffs_say_resnet = max_rsa.max_compactness_difference(compact_categories, compactness, nb_categories, listcat, models = ['saycam', 'resnet'], nb_max_compactness = 20)

results_say_resnet = find_max_dissimilarity_images(
        RDMs, ['saycam', 'resnet'], labels_say_resnet[:10], 50,
        images_per_subset=4)

In [ ]:
similarity_dict_say_resnet = compute_sub_rdm_similarity(results_say_resnet, RDMs, ['saycam', 'resnet'], labels_say_resnet[:10])
imagelist = [img.replace('/raid/shared/datasets/visoin/', '/home/alban/Documents/') for img in imagelist]
images, paths = display_low_similarity_images(imagelist, similarity_dict_say_resnet['selected_indices'], n_images=40,
                                 grid_cols=8, figsize=(20, 10), save_path='figures/images_inverse_Fisher_descriminant_saycam_resnet.png')

In [ ]:
nb_categories = len(listcat)
labels_say_resnet, sortedmaxdiffcats_say_resnet, maxdiffs_say_resnet = max_rsa.max_compactness_difference(compact_categories, compactness, nb_categories, listcat, models = ['saycam', 'resnet'], nb_max_compactness = 20)

results_say_resnet = find_max_dissimilarity_images(
        RDMs, ['saycam', 'resnet'], labels_say_resnet[:10], 50,
        images_per_subset=4)

In [ ]:
similarity_dict_say_resnet = compute_sub_rdm_similarity(results_say_resnet, RDMs, ['saycam', 'resnet'], labels_say_resnet[:10])
imagelist = [img.replace('/raid/shared/datasets/visoin/', '/home/alban/Documents/') for img in imagelist]
images, paths = display_low_similarity_images(imagelist, similarity_dict_say_resnet['selected_indices'], n_images=40,
                                 grid_cols=8, figsize=(20, 10), save_path='figures/images_inverse_Fisher_descriminant_saycam_resnet.png')

In [ ]:
sim_samples_say_resnet, indices_used_say_resnet = sample_rdm_pairs(RDMs['saycam'], RDMs['resnet'], n_samples=1000000, subset_size=40, batch_size=1000, seed=None)

In [ ]:
hist, bin_edges = np.histogram(sim_samples_say_resnet, 100)
plt.bar(bin_edges[:-1],hist/max(hist), width = bin_edges[1] - bin_edges[0], align = 'edge')
plt.vlines(similarity_dict_say_resnet['similarity'], 0, 1, 'green', label = 'Subset_selected')
plt.vlines(np.mean(sim_samples_say_resnet), 0, 1, 'r', label = 'Average')
plt.legend()
plt.xlabel('Similarity')
plt.ylabel('Density')
plt.show()
perc = np.sum(sim_samples_say_resnet< similarity_dict_say_resnet['similarity'])/len(sim_samples_say_resnet)
print(f'Number of Similarity samples with a lower similarity than our method: {int(perc*len(sim_samples))}')